In [6]:
! pip install SPARQLWrapper

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 5.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.7 MB/s eta 0:00:00


In [7]:
import json
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON
from requests import get
import re
import time
import asyncio
import aiohttp
import requests
from tqdm import tqdm
from tqdm.asyncio import tqdm_asyncio

# Round1

In [ ]:
cta_file = './data/Dataset/Dataset/Round1_T2D/gt/CTA_Round1_gt.csv'

mapping = {
    "LOC": [
        "Place", "PopulatedPlace", "City", "Country", "Region", "Mountain", "Island", "Lake", "River",
        "Park", "Building", "HistoricPlace", "Monument", "Bridge", "Road", "Airport"
    ],
    "PERS": [
        "Person", "Artist", "Athlete", "Politician", "Scientist", "Writer", "Actor", "Musician", "MilitaryPerson",
        "Religious", "Royalty", "Criminal"
    ],
    "ORG": [
        "Organisation", "Company", "EducationalInstitution", "PoliticalParty", "SportsTeam", "Non-ProfitOrganisation",
        "GovernmentAgency", "ReligiousOrganisation", "Band", "Library", "Museum", "Hospital", "University", "TradeUnion"
    ]
}

# Create reverse mapping
reverse_mapping = {v: k for k, values in mapping.items() for v in values}

# Define function to map df[2] values to their categories
def map_class_to_category(class_name):
    return reverse_mapping.get(class_name, "OTHERS")

# Apply the function and create the 'key' column
cta_keys = {}
df = pd.read_csv(cta_file, header=None)
type = df[2].astype(str).str.split('/').str[-1]
df["category"] = type.apply(map_class_to_category)
cta_keys["key"] = (df[0] + " " + df[1].astype('str'))
cta_keys["category"] = df["category"]


In [ ]:
df = pd.DataFrame(cta_keys)

# Save DataFrame to a CSV file
df.to_csv('./data/GT/Round1.csv', index=False)

# Round3

In [ ]:
cta_file = './data/Dataset/Dataset/Round3_2019/gt/CTA_Round3_gt.csv'

# Apply the function and create the 'key' column
cta_keys = {}
df = pd.read_csv(cta_file, header=None)
category_list = []

for row_idx in range(df.shape[0]):
    col_idx = 2
    while True:
        try:
            if pd.isna(df.iloc[row_idx,col_idx]):
                category_list.append("OTHERS")
                break
            urls = df.iloc[row_idx,col_idx].split(' ')
        except IndexError as e:
            category_list.append("OTHERS")
            break
        
        #print(f"{df.iloc[row_idx,0]}->{cell_urls} @ {row_idx},{col_idx}")
        find = False
        for url in urls:
            type = url.split('/')[-1] 
            if type == "Person":
                category_list.append("PER")
                find = True
                break
            elif type == "Location":
                category_list.append("LOC")
                find = True
                break
            elif type == "Organisation":
                category_list.append("ORG")
                find = True
                break
        if find:
            break
        
        col_idx += 1



df["category"] = category_list
cta_keys = {}
cta_keys["key"] = (df[0] + " " + df[1].astype('str'))
cta_keys["category"] = df["category"]


In [ ]:
df = pd.DataFrame(cta_keys)

# Save DataFrame to a CSV file
df.to_csv('./data/GT/Round3.csv', index=False)

# 2T_Round4

In [84]:
file_path = './data/output_gemma2/output/classified_2T.txt'
macroType_mapping = pd.read_csv(file_path)

In [85]:
cta_file = './data/Dataset/Dataset/2T_Round4/gt/cta.csv'
url = 'https://lamapi.hel.sintef.cloud/entity/labels?token=lamapi_demo_2023'

# Define the headers
headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json'
}

# Asynchronous function to get item root class
async def get_item_root(session, id_list):
    id_to_root_class = {}
    for el in id_list:
        match = re.search(r'Q(\d+)$', el)
        
        if match:
            data = {
                'json': [match[0]]
            }
        
            json_data = json.dumps(data)
            
            # Send POST request asynchronously
            async with session.post(url, headers=headers, data=json_data) as response:
                try:
                    result = await response.json()
                    root_class = result[match[0]]['labels']['en']
                    return root_class
                except:
                    pass
    return None

# Function to map categories to macroTypes
def map_macroTypes(category, macroType_mapping_df):
    # Find the macroType corresponding to the category (label)
    row = macroType_mapping_df[macroType_mapping_df['label'] == category]
    if not row.empty:
        return row['class'].values[0]
    return 'UNKNOWN'  # In case category does not match any

# Function to process the dataframe asynchronously with progress bar
async def process_dataframe(macroType_mapping_df):
    # Read the CSV file
    df = pd.read_csv(cta_file, header=None)
    
    root_categories = []
    async with aiohttp.ClientSession() as session:
        # Create a list of tasks to fetch root classes
        tasks = []
        for urls in df[2]:
            tmp = [url.split('/')[-1] for url in urls.split(" ")]
            tasks.append(get_item_root(session, tmp))
        
        # Use tqdm_asyncio to track the progress of tasks
        root_categories = await tqdm_asyncio.gather(*tasks, desc="Fetching Root Classes", total=len(tasks))
    
    # Assign the root categories to the dataframe
    df["category"] = root_categories
    
    # Map the categories to macroTypes and add it as a new column
    df['macroTypes'] = df['category'].apply(lambda category: map_macroTypes(category, macroType_mapping_df))
     
    # Creating the cta_keys structure
    cta_keys = pd.DataFrame()
    cta_keys["key"] = df[0].astype(str) + " " + df[1].astype(str)
    cta_keys["category"] = df["category"]
    cta_keys["macroTypes"] = df["macroTypes"]
    
    return cta_keys

# Entry point to run the async function
if __name__ == "__main__":
    df = await(process_dataframe(macroType_mapping))


Fetching Root Classes: 100%|██████████| 540/540 [00:03<00:00, 143.96it/s]


In [86]:
df = pd.DataFrame(df)

# Save DataFrame to a CSV file
df.to_csv('./data/GT/2T_Round4.csv', index=False)

# Round4

In [ ]:
# Define the combined macroType mapping
macroType_mapping = {
    'LOC': [
        'neighborhood of Brazil', 'provincial park in Saskatchewan', 'airport railway station',
        'city of regional significance of Ukraine', 'national wildlife area of Canada',
        'California state park', 'city palace', 'municipality of Sweden', 'department of El Salvador',
        'township of Illinois', 'village of New York', 'borough of Alaska', 'town in Romania',
        'city district of Brazil', 'township of New Jersey', 'county of Missouri', 
        'county of South Carolina', 'county of Florida', 'county of North Carolina', 
        'municipality of Latvia', 'town of Virginia', 'township of Missouri', 
        'town divided by border', 'department of transportation', 'historic county of England',
        'natural region of France', 'region of Italy', 'province of Papua New Guinea', 
        'district of Nepal', 'district of the German Democratic Republic', 'autonomous prefecture',
        'township of New Jersey', 'municipality of Guatemala', 'municipality of Switzerland',
        'region of Kazakhstan', 'town', 'lake', 'district of Iran', 'commune of Benin',
        'hamlet in Alberta', 'municipality in Germany', 'municipality of Puerto Rico', 
        'county of Oregon', 'administrative territorial entity of Russia', 'national park of Quebec', 
        'county of Kansas', 'county of Indiana', 'town of Newfoundland and Labrador', 
        'district of Malawi', 'village in Indonesia', 'region of the Philippines',
        'townland', 'rural area', 'village of Poland', 'district of New Zealand', 
        'village/town/city in Lebanon', 'provincial capital', 'local municipality',
        'Commonwealth War Graves Commission maintained memorial', 'roadside station in Japan', 'township of Kansas',
        'cemetery cross', 'viaduct', 'space station', 'bullring', 'Lok Sabha constituency',
        'flag of a country subdivision', 'mountain pass', 'coffeehouse', 'scale model', 
        'Maryland state park', 'city block', 'river basin district', 'municipality of Cuba',
        'oasis', 'township municipality', 'urban park', 'national park of Brazil', 
        'national preserve', 'geographical feature', 'village in Belarus', 'city municipality',
        'commune of Chile', 'township of China', 'district capital', 'region of Brazil',
        'local government area of Queensland', 'city or town in Armenia', 'village of Japan',
        'township of North Dakota', 'town of Pennsylvania', 'highly urbanized city', 
        'neighbourhood of Helsinki', 'urban commune of Morocco', 'historic district',
        'municipality of Greece', 'capital city', 'district of Japan', 'community area in Chicago',
        'provincial city', 'township of Nebraska', 'region of Madagascar', 'rural municipality of Poland',
        'county of California', 'region of France', 'associated commune', 'rural settlement in Russia',
        'avenue', 'urban-type settlement in Russia', 'moon of Uranus', 'warehouse', 'city walls',
        'location of burial', 'district of Mongolia', 'parish municipality', 'town of Portugal',
        'town of Japan', 'city in Colombia', 'region of Peru', 'county of Alabama', 
        'province of Morocco', 'natural monument', 'capital of prefecture', 'fjord', 'village municipality of Quebec',
        'census-designated place in the United States', 'township of Indiana', 'township of Arkansas', 
        'district municipality', 'district of India', 'village in India', 'municipality of Niger', 
        'province of Sweden', 'parish of Louisiana', 'urban council of Ukraine', 'central business district',
        'island of Portugal', 'lower-tier municipality', 'city of Portugal', 'city of the Philippines',
        'community school', 'municipality of the Philippines', 'community development block in India',
        'administrative territorial entity of Canada', 'natural park', 'community of Cyprus Republic',
        'exclave', 'harbor', 'sconce', 'city in the state of New York', 'city of Indonesia', 
        'district of Costa Rica', 'county of Kentucky', 'municipality of Japan', 'cultural heritage', 
        'historical country', 'political territorial entity', 'unitary authority area in England', 'state of Nigeria',
        'district of Uganda', 'district of Belarus', 'district of Serbia', 'municipality of Mexico', 
        'urban municipality in Germany', 'village municipality', 'city in Armenia'
    ],
    'ORG': [
        'bar association', 'film production company', 'film studio', 'heavy metal band',
        'auto racing team', 'professional sports team', 'political party in Germany', 
        'dance', 'research council', 'foundation school', 'commercial organization', 
        'national badminton team', 'medical association', 'federal electoral district of Germany', 
        'public library', 'badminton association', 'sports club', 'television series season',
        'university press', 'television series episode', 'association football team',
        'weekly newspaper', 'cultural center', 'academy', 'university and college sports club', 
        'programming tool', 'conference paper', 'professional association', 'sports competition',
        'carnival', 'federation', 'national library',
        'piano sonata', 'film award', 'music museum', 'music venue', 'restaurant chain',
        'educational institution', 'medical society', 'academic library', 'government building',
        'advocacy group', 'commercial bank', 'university in France', 'association football team season',
        'musical release', 'professional sports team', 'multinational corporation', 'television network',
        'social media platform', 'music festival', 'research university', 'sports league',
        'international organization', 'cultural institution', 'nonprofit organization',
        'news agency', 'concert hall', 'Lyceum', 'youth organization', 'music organization', 
        'political party', 'theatre company', 'football club', 'futsal team', 'diplomatic mission', 
        'armed forces', 'sports venue', 'association under the French law of 1901', 
        'military occupation', 'commercial organization', 'historical motorcycle manufacturer',
        'cable channel', 'Boston Open Badminton Championships', 'LEN European Aquatics Championships',
        'Czech Open', 'Italian Grand Prix', 'Baltic state', 'Christian denomination', 'political party',
        'Nobel family', 'music school', 'college', 'restaurant chain', 'film festival', 
        'nonprofit organization', 'sports club', 'federal agency of Germany', 'sports park', 
        'broadcaster', 'association football league', 'music genre', 'local government area of Nigeria', 
        'political organization', 'national trade union center', 'international organization', 
        'media company', 'government', 'department of Ivory Coast', 'academic journal article',
        'sports league', 'fictional organization', 'regional council of Israel', 'academic department',
        'sports team', 'amateur sports organization', 'musical ensemble', 'noble family', 
        'historically black colleges and universities', 'historical organization', 'police station',
        'university building', 'group of sculptures', 'transport service itinerary', 'community of Cyprus Republic'
    ],
    'PERS': [
        'family', 'profession', 'married couple', 'female given name',
        'Japanese television series', 'fictional detective', 'anime film', 'superhero film character',
        'boy band', 'mythological character', 'famous artist', 'author', 'historical figure',
        'young stellar object candidate', 'individual time trial', 'Homo sapiens',
        'corporate title', 'human biblical figure', 'fictional deity', 'political organization',
        'sports discipline', 'art movement', 'musical profession', 'military rank'
    ],
    'OTHERS': [
        'flammable solid', 'globular cluster', 'emakimono', 'air-to-air missile', 
        'report', 'fragment', 'stream', 'prestressed concrete bridge', 'show cave', 
        'fictional humanoid species', 'imperative programming language', 'weapon family', 
        'transcript fusion', 'productivity software', 'photograph album', 'specialty channel', 
        'chemical compound', 'flying-type Pokémon', 'bug-type Pokémon', 'artificial lake', 
        'designated intractable/rare disease', 'region', 'urban area', 'suburb of Perth',
        'trunk road', 'drama television series', 'supervillain', 'comic book', 
        'characteristic', 'courage award', 'space program', 'film festival', 
        'tapestry', 'medication', 'economic sector', 'color', 'metaphor',
        'LP record', 'short story collection', 'rock formation', 'explosion', 'bus type', 
        'letter', 'compilation soundtrack album', 'isotope', 'cartoonist', 'bridge',
        'mushroom', 'commercially significant film', 'nonprofit sector', 'political region', 
        'marathon', 'figure of speech', 'poetry collection', 'historical period', 
        'fountain', 'glacier', 'religious text', 'set of historical events', 'political figure',
        'public park', 'forest', 'city park', 'recreational vehicle', 'pet species', 
        'rural town', 'historic place', 'historical event', 'historic region', 
        'religious building', 'current country', 'floral arrangement', 'character type',
        'theoretical concept', 'local newspaper', 'acronym', 'neighborhood of Sweden', 
        'national sporting event', 'social construct', 'floral species', 'national park',
        'climate zone', 'emotional concept', 'artistic movement', 'theory', 
        'mechanical system', 'expression', 'collection', 'statistical population',
        'partnership', 'outdoor venue', 'social organization', 'assembly', 
        'food item', 'fusion cuisine', 'economic classification', 'massive urban area',
        'species', 'goddess', 'natural disaster', 'cultural movement', 'fire prevention method',
        'historical location', 'classic rock song', 'classical music composition',
        'historical region', 'role-playing game', 'physical feature', 'hot air balloon',
        'supernatural entity', 'piece of writing', 'documentary film', 'discussion forum',
        'individual film', 'software project', 'mountain', 'bicycle category', 
        'scientific research', 'radio show', 'book genre', 'religious movement', 
        'common area', 'university publication', 'mountain range', 'urban area', 
        'dispute', 'artistic period', 'mode of transportation', 'road', 'venue',
        'election campaign', 'artwork', 'journal article', 'acquired trait', 
        'scientific publication', 'form of government', 'literary genre'
    ]
}


In [ ]:
cta_file = './data/Dataset/Dataset/Round4_2020/gt/cta.csv'
url = 'https://lamapi.hel.sintef.cloud/entity/labels?token=lamapi_demo_2023'

# Define the headers
headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json'
}

# Asynchronous function to get item root class
async def get_item_root(session, id_list):
    id_to_root_class = {}
    for el in id_list:
        match = re.search(r'Q(\d+)$', el)
        
        if match:
            data = {
                'json': [match[0]]
            }
        
            json_data = json.dumps(data)
            
            # Send POST request asynchronously
            async with session.post(url, headers=headers, data=json_data) as response:
                try:
                    result = await response.json()
                    root_class = result[match[0]]['labels']['en']
                    return root_class
                except:
                    pass
    return None

# Function to map categories to macroTypes
def map_macroTypes(category):
    for macroType, categories in macroType_mapping.items():
        if category in categories:
            return macroType

    return 'UNKNOWN'  # In case category does not match any

# Function to process the dataframe asynchronously with progress bar
async def process_dataframe():
    # Read the CSV file
    df = pd.read_csv(cta_file, header=None)
    
    root_categories = []
    async with aiohttp.ClientSession() as session:
        # Create a list of tasks to fetch root classes
        tasks = []
        for urls in df[2]:
            tmp = [url.split('/')[-1] for url in urls.split(" ")]
            tasks.append(get_item_root(session, tmp))
        
        # Use tqdm_asyncio to track the progress of tasks
        root_categories = await tqdm_asyncio.gather(*tasks, desc="Fetching Root Classes", total=len(tasks))
    
    # Assign the root categories to the dataframe
    df["category"] = root_categories
    
    # Map the categories to macroTypes and add it as a new column
    df['macroTypes'] = df['category'].apply(map_macroTypes)
    
    # Creating the cta_keys structure
    cta_keys = pd.DataFrame()
    cta_keys["key"] = df[0].astype(str) + " " + df[1].astype(str)
    cta_keys["category"] = df["category"]
    cta_keys["macroTypes"] = df["macroTypes"]
    
    return cta_keys

# Entry point to run the async function
if __name__ == "__main__":
    df = await(process_dataframe())


In [ ]:
df = pd.DataFrame(df)

In [ ]:
df

In [ ]:
from collections import Counter

macro_type_counts = Counter(df['macroTypes'])

# Print the counts
print(macro_type_counts)

In [ ]:
pd.DataFrame(cta_keys)

In [ ]:
cta_file = './data/Dataset/Dataset/Round4_2020/gt/cta.csv'

def get_item_root(id_list):    
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
    id_to_root_class = {}
    
    for el in tqdm(id_list, desc="Processing IDs"):
        if el not in id_to_root_class:
            query = f"""
            SELECT ?instanceClass ?instanceClassLabel WHERE {{
              wd:{el} wdt:P31 ?instanceClass .
              SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE]" }}
            }}
            """
            
            # Set the query and request JSON response
            sparql.setQuery(query)
            sparql.setReturnFormat(JSON)
            #time.sleep(0.5)
            
            try:
                results = sparql.query().convert()
                if len(results["results"]["bindings"]) > 0:
                    inst_item = int(results["results"]["bindings"][0]['instanceClassLabel']['value'][1:])
                    if inst_item in geolocation_subclass:
                        id_to_root_class[el] = "LOC"
                    elif inst_item in organization_subclass:
                        id_to_root_class[el] = "ORG"
                    elif inst_item == 5 or el == "Q5":
                        id_to_root_class[el] = "PERS"
                    else:
                        id_to_root_class[el] = "OTHERS"
                else:
                    id_to_root_class[el] = "None"
            except Exception as e:
                print(f"Error processing {el}: {e}")
                time.sleep(0.5)
                id_to_root_class[el] = "None"          
    
    return id_to_root_class

# Apply the function and create the 'key' column
root_classes = []
df = pd.read_csv(cta_file, header=None)
ids = [url.split('/')[-1] for url in df[2]]

root_classes = get_item_root(ids)

# Map root classes to categories
root_categories = []
for el in ids:
    try:
        root_categories.append(root_classes[el])
    except:
        pass

df["category"] = root_categories
cta_keys = {}
cta_keys["key"] = (df[0] + " " + df[1].astype('str'))
cta_keys["category"] = df["category"]

In [ ]:
df = pd.DataFrame(cta_keys)

# Save DataFrame to a CSV file
df.to_csv('./data/GT/Round4.csv', index=False)

# HardTableR2

In [78]:
file_path = './data/output_gemma2/output/classified_HardTableR2.txt'
macroType_mapping = pd.read_csv(file_path)

In [79]:
cta_file = './data/Dataset/Dataset/HardTablesR2/gt/cta.csv'
url = 'https://lamapi.hel.sintef.cloud/entity/labels?token=lamapi_demo_2023'

# Define the headers
headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json'
}

# Asynchronous function to get item root class
async def get_item_root(session, id_list):
    id_to_root_class = {}
    for el in id_list:
        match = re.search(r'Q(\d+)$', el)
        
        if match:
            data = {
                'json': [match[0]]
            }
        
            json_data = json.dumps(data)
            
            # Send POST request asynchronously
            async with session.post(url, headers=headers, data=json_data) as response:
                try:
                    result = await response.json()
                    root_class = result[match[0]]['labels']['en']
                    return root_class
                except:
                    pass
    return None

# Function to map categories to macroTypes
def map_macroTypes(category, macroType_mapping_df):
    # Find the macroType corresponding to the category (label)
    row = macroType_mapping_df[macroType_mapping_df['label'] == category]
    if not row.empty:
        return row['class'].values[0]
    return 'UNKNOWN'  # In case category does not match any

# Function to process the dataframe asynchronously with progress bar
async def process_dataframe(macroType_mapping_df):
    # Read the CSV file
    df = pd.read_csv(cta_file, header=None)
    
    root_categories = []
    async with aiohttp.ClientSession() as session:
        # Create a list of tasks to fetch root classes
        tasks = []
        for urls in df[2]:
            tmp = [url.split('/')[-1] for url in urls.split(" ")]
            tasks.append(get_item_root(session, tmp))
        
        # Use tqdm_asyncio to track the progress of tasks
        root_categories = await tqdm_asyncio.gather(*tasks, desc="Fetching Root Classes", total=len(tasks))
    
    # Assign the root categories to the dataframe
    df["category"] = root_categories
    
    # Map the categories to macroTypes and add it as a new column
    df['macroTypes'] = df['category'].apply(lambda category: map_macroTypes(category, macroType_mapping_df))
     
    # Creating the cta_keys structure
    cta_keys = pd.DataFrame()
    cta_keys["key"] = df[0].astype(str) + " " + df[1].astype(str)
    cta_keys["category"] = df["category"]
    cta_keys["macroTypes"] = df["macroTypes"]
    
    return cta_keys

# Entry point to run the async function
if __name__ == "__main__":
    df = await(process_dataframe(macroType_mapping))


Fetching Root Classes: 100%|██████████| 2191/2191 [00:03<00:00, 572.17it/s]


In [69]:

from collections import Counter

macro_type_counts = Counter(df['macroTypes'])

# Print the counts
print(macro_type_counts)

Counter({'OTHER': 1054, 'LOCATION': 710, 'ORGANIZATION': 189, 'UNKNOWN': 123, 'PERSON': 115})


In [80]:
df = pd.DataFrame(df)

# Save DataFrame to a CSV file
df.to_csv('./data/GT/HTR2.csv', index=False)

# HardTableR3

In [81]:
file_path = './data/output_gemma2/output/classified_HardTableR3.txt'
macroType_mapping = pd.read_csv(file_path)

In [82]:
cta_file = './data/Dataset/Dataset/HardTablesR3/gt/cta.csv'
url = 'https://lamapi.hel.sintef.cloud/entity/labels?token=lamapi_demo_2023'

# Define the headers
headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json'
}

# Asynchronous function to get item root class
async def get_item_root(session, id_list):
    id_to_root_class = {}
    for el in id_list:
        match = re.search(r'Q(\d+)$', el)
        
        if match:
            data = {
                'json': [match[0]]
            }
        
            json_data = json.dumps(data)
            
            # Send POST request asynchronously
            async with session.post(url, headers=headers, data=json_data) as response:
                try:
                    result = await response.json()
                    root_class = result[match[0]]['labels']['en']
                    return root_class
                except:
                    pass
    return None

# Function to map categories to macroTypes
def map_macroTypes(category, macroType_mapping_df):
    # Find the macroType corresponding to the category (label)
    row = macroType_mapping_df[macroType_mapping_df['label'] == category]
    if not row.empty:
        return row['class'].values[0]
    return 'UNKNOWN'  # In case category does not match any

# Function to process the dataframe asynchronously with progress bar
async def process_dataframe(macroType_mapping_df):
    # Read the CSV file
    df = pd.read_csv(cta_file, header=None)
    
    root_categories = []
    async with aiohttp.ClientSession() as session:
        # Create a list of tasks to fetch root classes
        tasks = []
        for urls in df[2]:
            tmp = [url.split('/')[-1] for url in urls.split(" ")]
            tasks.append(get_item_root(session, tmp))
        
        # Use tqdm_asyncio to track the progress of tasks
        root_categories = await tqdm_asyncio.gather(*tasks, desc="Fetching Root Classes", total=len(tasks))
    
    # Assign the root categories to the dataframe
    df["category"] = root_categories
    
    # Map the categories to macroTypes and add it as a new column
    df['macroTypes'] = df['category'].apply(lambda category: map_macroTypes(category, macroType_mapping_df))
     
    # Creating the cta_keys structure
    cta_keys = pd.DataFrame()
    cta_keys["key"] = df[0].astype(str) + " " + df[1].astype(str)
    cta_keys["category"] = df["category"]
    cta_keys["macroTypes"] = df["macroTypes"]
    
    return cta_keys

# Entry point to run the async function
if __name__ == "__main__":
    df = await(process_dataframe(macroType_mapping))


Fetching Root Classes: 100%|██████████| 7207/7207 [00:10<00:00, 662.70it/s] 


In [83]:
df = pd.DataFrame(df)

# Save DataFrame to a CSV file
df.to_csv('./data/GT/HTR3.csv', index=False)

# Promt LLM

In [ ]:
string_promt = "classify each of the items according to the following macro types ORG, LOC, PERS, OTHERS. an item can belong only to one class with the following python dictionary format CSV with columns (label_name, macroTypes) es. Docklands Light Railway station, LOC"

# Read a txt file where items are separated by commas
file_path = './data/GT/types_list/Round4.txt'

# Read the file and split the line into a list of items
with open(file_path, 'r') as file:
    # Read the entire content as a single line
    content = file.read()
    
    # Remove single quotes from the content
    content = content.replace("'", "")
    
    # Split the content by commas and strip any extra spaces
    items = [item.strip() for item in content.split(',')]

batch_size = 100

data = []

for i in range(0, len(items), batch_size):
    batch = items[i:i + batch_size]
    prompt = f"{string_prompt} {'; '.join(batch)}."
    
    ## .... LLM
    # format output = [(), (), ...]
    
    data = data + output


df = pd.DataFrame(data, columns=['type', 'macroType'])

# LLM macroType evaluation

In [52]:
gemma_file_path = './data/output_gemma2/output/classified_Round4.txt'
mistral_file_path = './data/output_mistral/output2/classified_Round4.txt'

gemma = pd.read_csv(gemma_file_path)
mistral = pd.read_csv(mistral_file_path)

# Assume the column containing entities is named 'entity'; adjust as necessary
merged_df = pd.merge(gemma, mistral, on='label', suffixes=('_gemma', '_mistral'))

# Step 2: Find where classes are different
differences = merged_df[merged_df['class_gemma'] != merged_df['class_mistral'].str.split().str[0]]

# Display the differences
print("\nDifferences in Class Labels:")
(differences)
differences.to_csv('differences.csv', index=False)


Differences in Class Labels:


In [53]:
gemma_file_path = './data/output_gemma2/output/classified_2T.txt'
mistral_file_path = './data/output_mistral/output2/classified_2T.txt'

gemma = pd.read_csv(gemma_file_path)
mistral = pd.read_csv(mistral_file_path)

# Assume the column containing entities is named 'entity'; adjust as necessary
merged_df = pd.merge(gemma, mistral, on='label', suffixes=('_gemma', '_mistral'))

# Step 2: Find where classes are different
differences = merged_df[merged_df['class_gemma'] != merged_df['class_mistral'].str.split().str[0]]

# Display the differences
print("\nDifferences in Class Labels:")
(differences)
differences.to_csv('./data/output_gemma2/differences_2T.csv', index=False)



Differences in Class Labels:


In [54]:
gemma_file_path = './data/output_gemma2/output/classified_HardTableR2.txt'
mistral_file_path = './data/output_mistral/output2/classified_HardTableR2.txt'

gemma = pd.read_csv(gemma_file_path)
mistral = pd.read_csv(mistral_file_path)

# Assume the column containing entities is named 'entity'; adjust as necessary
merged_df = pd.merge(gemma, mistral, on='label', suffixes=('_gemma', '_mistral'))

# Step 2: Find where classes are different
differences = merged_df[merged_df['class_gemma'] != merged_df['class_mistral'].str.split().str[0]]

# Display the differences
print("\nDifferences in Class Labels:")
(differences)
differences.to_csv('./data/output_gemma2/differences_HardTableR2.csv', index=False)



Differences in Class Labels:


In [55]:
gemma_file_path = './data/output_gemma2/output/classified_HardTableR3.txt'
mistral_file_path = './data/output_mistral/output2/classified_HardTableR3.txt'

gemma = pd.read_csv(gemma_file_path)
mistral = pd.read_csv(mistral_file_path)

# Assume the column containing entities is named 'entity'; adjust as necessary
merged_df = pd.merge(gemma, mistral, on='label', suffixes=('_gemma', '_mistral'))

# Step 2: Find where classes are different
differences = merged_df[merged_df['class_gemma'] != merged_df['class_mistral'].str.split().str[0]]

# Display the differences
print("\nDifferences in Class Labels:")
(differences)
differences.to_csv('./data/output_gemma2/differences_HardTableR3.csv', index=False)



Differences in Class Labels:
